In [ ]:
!pip install git+https://github.com/CarperAI/trlx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CarperAI/trlx to /tmp/pip-req-build-5aj728gl
  Running command git clone --filter=blob:none --quiet https://github.com/CarperAI/trlx /tmp/pip-req-build-5aj728gl
  Resolved https://github.com/CarperAI/trlx to commit 355c9741f2e606de796f5c6f9b682f7dd00f97c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.3/779.3 kB 61.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 81.2 MB/s eta 0

In [ ]:
pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import trlx
from trlx.data.default_configs import default_ilql_config
config = default_ilql_config().evolve(train=dict(batch_size=32, total_steps=300), from_tf=True)

In [ ]:
!pip install trueskill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18048 sha256=73643e06c41532808ad5d17448dafa0db2bf93276c3981fe8747d23bbc73cf8d
  Stored in directory: /root/.cache/pip/wheels/b9/4f/29/c79f0a2956775524c7a23638ac2b6fbb516c680f8e5eed9b53
Successfully built trueskill


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import trueskill

# Initialize TrueSkill rating system parameters
trueskill.setup(mu=25.0, sigma=8.333, beta=4.167, tau=0.0833, draw_probability=0.0)

# Load the original dataframe
df = pd.read_csv('/content/drive/MyDrive/csv/my_data.csv')

# Create a list of unique poems
poems = df["Poem"].unique()

# Create a dictionary to store the TrueSkill ratings for each poem
ratings = {poem: trueskill.Rating() for poem in poems}

# Create a new dataframe to store the TrueSkill ratings as float values
ratings_df = pd.DataFrame({"Poem": poems, "TrueSkill Rating": [r.mu for r in ratings.values()]})

# Create a dictionary to store the index of each poem in the ratings_df DataFrame
ratings_df_indices = {poem: i for i, poem in enumerate(poems)}

problematic_rows = []

# Define a function to update TrueSkill ratings for a group
def update_group_ratings(group):
    try:
        # Extract the poems and rankings from the group
        ranked_poems = group.sort_values("Ranking")["Poem"].tolist()

        # Update the TrueSkill ratings based on the sorted ranking
        new_ratings = trueskill.rate([[ratings[ranked_poems[0]]], [ratings[ranked_poems[1]]], [ratings[ranked_poems[2]]], [ratings[ranked_poems[3]]]], ranks=[0, 1, 2, 3])

        # Flatten the new_ratings list and update the ratings dictionary
        new_ratings = [item for sublist in new_ratings for item in sublist]
        ratings.update(zip(ranked_poems, new_ratings))

        # Update the TrueSkill ratings in the ratings_df DataFrame as float values
        for j in range(4):
            idx = ratings_df_indices[ranked_poems[j]]
            ratings_df.at[idx, "TrueSkill Rating"] = ratings[ranked_poems[j]].mu
    except IndexError as e:
        print(f"Error in group: {e}")
        problematic_rows.append(group.index[0])

# Group the original dataframe by every 4 rows
groups = df.groupby(df.index // 4)

# Update TrueSkill ratings for each group
groups.apply(update_group_ratings)


""


In [ ]:
print(problematic_rows)

[]


In [ ]:
df.drop(problematic_rows, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
# Save the modified dataframe to a new CSV file
df.to_csv('/content/drive/MyDrive/csv/my_data.csv', index=False)

In [ ]:
# Convert the poem column into a list
poems = ratings_df['Poem'].tolist()

# Use the original ratings directly
ratings = ratings_df['TrueSkill Rating'].values

In [ ]:
# Convert the poem column into a list
poems = ratings_df['Poem'].tolist()

# Get the original trueskill ratings as a NumPy array
original_ratings = ratings_df['TrueSkill Rating'].values

# Normalize and rescale the trueskill ratings
min_rating = original_ratings.min()
max_rating = original_ratings.max()
min_reward = -5
max_reward = 5

normalized_ratings = (original_ratings - min_rating) / (max_rating - min_rating)
rescaled_ratings = min_reward + (max_reward - min_reward) * normalized_ratings

# Now, you can use 'poems' and 'rescaled_ratings' for further processing


Trlx uses [wandb](https://wandb.ai/) to log results. Make sure to set up an account and use your token to authenticate when prompted after executing the cell below.

In [ ]:
starts = pd.read_csv('/content/drive/MyDrive/csv/starts.csv')
startlist = starts['short_content'].tolist()

In [ ]:
model = trlx.train(
    "Yorth/gpt2_medium_poetry",
    config=config,
    samples=poems,
    rewards=rescaled_ratings
).model

[RANK 0] Initializing model: Yorth/gpt2_medium_poetry


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[RANK 0] Collecting rollouts
[RANK 0] Logging sample example


                                                Sample Example                                                
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Prompt        ┃ Response                                                              ┃ Reward             ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ <|endoftext|> │ The compassion for those in needful words shall fall                  │ 0.3183089209723411 │
│               │ Before the brazen gates of hell,                                      │                    │
│               │ And in the caverns of the dusky wood,--                               │                    │
│               │ In such a woful place he shall be clad, and shod,.--POPE<|endoftext|> │                    │
└───────────────┴───────────────────────────────────────────────────────────────────────┴────────────────────┘

...

                         Evaluation #2                         
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt ┃ output                                             ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        │ Lives of great men all remind us of our death.     │
│        │ For how could he be born but to forgive,           │
│        │ Who dares, and doth he speak, the fool, who dares? │
│        │ To his own self however we are                     │
├────────┼────────────────────────────────────────────────────┤
│        │ As the immense dew of Florida Brings,              │
│        │ Or St. George's triumph, or St. James's lust,--    │
│        │ All this and more of this most generous dower,--   │
│        │                                                    │
├────────┼────────────────────────────────────────────────────┤
│        │ As virtuous men pass mildly away And I to you      │
│        │ Little am I of those two kings I loved             │
│        │ And loved of one more. I love the king             │
│        │ Who loved my life and that he loved the world.     │
│        │                                                    │
│        │                                                    │
│        │                                                    │
│        │ Entered according                                  │
└────────┴────────────────────────────────────────────────────┘

[RANK 0] Saving intermediate checkpoint into ckpts/checkpoint_300
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/1]:   0%|          | 0/1 [00:00<?, ?it/s]

[RANK 0] Summarizing evaluation


                           Evaluation #3                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt ┃ output                                                  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        │ There is a garden in her face                           │
│        │ Where roses blow.                                       │
│        │ But in the wind there is no dew                         │
│        │ Where violets blow.                                     │
│        │                                                         │
│        │                                                         │
│        │                                                         │
│        │ All up and over the garden ways,                        │
│        │ Her feet have trod a mile with you                      │
├────────┼─────────────────────────────────────────────────────────┤
│        │ Weret aught to me I bore the breath of life,            │
│        │ And not in boyish service, that for gold,--             │
│        │ I yearned for a young husband's tender wife,--the wife. │
│        │ But I, who know the (                                   │
├────────┼─────────────────────────────────────────────────────────┤
│        │ Ezra Pound "Canto XLV " from The."                      │
│        │ Cum barber shall be born in the MS.:                    │
│        │ His fame must be of use and wit                         │
│        │ No longer to the world confine.--WARTON.                │
│        │  The first                                              │
└────────┴─────────────────────────────────────────────────────────┘

In [ ]:
# Infer the trained model
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>')
output = model.generate(**tokenizer(["Let the bird of loudest lay On   "] * 16, return_tensors="pt").to(0))
tokenizer.batch_decode(output, skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Let the bird of loudest lay On     _Another\nLoves in the wily-flaxen ilex\nTo the tiny gnat that spins\nFrom the tiny that he loves',
 'Let the bird of loudest lay On     _Excepiss._\nWARTLEY & SONSON, _Exce genetia, atque in coeple\nNeque',
 'Let the bird of loudest lay On     _Ex._\nI asked a woman in my youth\nWho else had learned to love me, even me,\nWho learned with scorn the trit',
 'Let the bird of loudest lay On     "\nThat crested leg" on which the bird is found--\nBut it is all the same--it is to-day.\nThat little',
 'Let the bird of loudest lay On    its nest.\nI was a bird in the beating air, a bird, I know, who was\nMy joy to the birds and the clouds,',
 'Let the bird of loudest lay On   vernal skies.\nWhat a beauty that is in her eye!\nWhat a light, a sister-love in her eye!\nShe sees in me',
 'Let the bird of loudest lay On     [_Another note_]\nTo the bird of the morning--   [_Another note_]\nHe lifted up his voice,  [',
 'Let the bird of loudest lay On    vernal sk

In [ ]:
# Save the model locally
model.save_pretrained("gpt2-poetry-boooosted")

In [ ]:
# To upload the model to Hugging Face, login first
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Upload the model to <your_name>/gpt2-simulacra
from huggingface_hub import create_repo, HfApi

repo_id = create_repo("gpt2-simulacra", private=False, exist_ok=True).repo_id
HfApi().upload_folder(folder_path="gpt2-simulacra", repo_id=repo_id)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:2181 in create_repo            │
│                                                                                                  │
│   2178 │   │   │   # Testing purposes only.                                                      │
│   2179 │   │   │   # See https://github.com/huggingface/huggingface_hub/pull/733/files#r8206044  │
│   2180 │   │   │   json["lfsmultipartthresh"] = self._lfsmultipartthresh  # type: ignore         │
│ ❱ 2181 │   │   headers = self._build_hf_headers(token=token, is_write_action=True)               │
│   2182 │   │   r = get_session().post(path, headers=headers, json=json)                          │
│   2183 │   │                                                                                     │
│   2184 │   │   try:                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:4732 in _build_hf_headers      │
│                                                                                                  │
│   4729 │   │   if token is None:                                                                 │
│   4730 │   │   │   # Cannot do `token = token or self.token` as token can be `False`.            │
│   4731 │   │   │   token = self.token                                                            │
│ ❱ 4732 │   │   return build_hf_headers(                                                          │
│   4733 │   │   │   token=token,                                                                  │
│   4734 │   │   │   is_write_action=is_write_action,                                              │
│   4735 │   │   │   library_name=library_name or self.library_name,                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                    

In [ ]:
# Load the same model now stored on Hugging Face
from trlx.models.modeling_ilql import AutoModelForCausalLMWithILQLHeads
hf_model = AutoModelForCausalLMWithILQLHeads.from_pretrained(repo_id)